# Primal–Dual Interior Point Methods

This notebook implements three primal–dual interior-point algorithms for linear programming:

1. Central path with fixed step size and centering parameter.
2. Central path with adaptive step size and centering parameter.
3. Mehrotra predictor–corrector method.

The notebook includes several LP test problems, runs the algorithms, saves plots (objective vs iteration, central path, complementarity), and compares results with SciPy's `linprog`.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog
import json, os
import warnings
warnings.filterwarnings('ignore')

# Publication-quality settings (IEEE/ACM/NeurIPS style)
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif'],
    'font.size': 9,
    'axes.labelsize': 10,
    'axes.titlesize': 10,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'legend.fontsize': 8,
    'figure.dpi': 100,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    'savefig.pad_inches': 0.05,
    'lines.linewidth': 1.5,
    'axes.linewidth': 0.8,
    'grid.linewidth': 0.5,
    'legend.framealpha': 0.95,
    'legend.edgecolor': '#CCCCCC',
    'axes.grid': True,
    'grid.alpha': 0.3,
    'text.usetex': False
})

COLORS = {
    'fixed': '#0173B2',      # Blue
    'adaptive': '#DE8F05',   # Orange  
    'mehrotra': '#029E73',   # Green
    'scipy': '#CC78BC',      # Purple
    'tolerance': '#D55E00'   # Red-orange
}

MARKERS = {'fixed': 'o', 'adaptive': 's', 'mehrotra': '^'}

os.makedirs('../figures', exist_ok=True)

In [2]:
def make_standard_form(c, A_ineq=None, b_ineq=None):
    """Convert min c^T x s.t. A_ineq x <= b_ineq into standard form:
    min c^T x, A x = b, x >= 0 by adding slack variables.
    Returns (c_std, A, b) where x_std = [x; s] includes slack vars.
    """
    if A_ineq is None:
        return c, np.zeros((0,len(c))), np.zeros(0)
    m, n = A_ineq.shape
    A = np.hstack([A_ineq, np.eye(m)])
    c_std = np.hstack([c, np.zeros(m)])
    b = b_ineq.copy()
    return c_std, A, b

def initial_point(A, b, c, delta=1.0):
    """Heuristic initialization following Nocedal & Wright page 410"""
    n = len(c)
    m = A.shape[0]
    x = np.maximum(1.0, 0.1 * np.ones(n))
    s = np.maximum(1.0, 0.1 * np.ones(n))
    
    # Adjust x to satisfy Ax=b
    if m > 0:
        rhs = b - A.dot(x)
        try:
            AAT = A.dot(A.T)
            if np.linalg.cond(AAT) < 1e10:
                lam = np.linalg.solve(AAT, rhs)
                x = x + A.T.dot(lam)
        except Exception:
            pass
    
    y = np.zeros(m)
    x = np.maximum(x, 1e-3)
    s = np.maximum(s, 1e-3)
    return x, y, s

def compute_residuals(A, b, c, x, y, s):
    """Compute KKT residuals"""
    rc = A.T.dot(y) + s - c
    rb = A.dot(x) - b
    rxs = x * s
    return rc, rb, rxs

def mu_val(x, s):
    """Compute duality measure"""
    return (x.dot(s)) / len(x)

In [3]:
def make_standard_form(c, A_ineq=None, b_ineq=None):
    """Convert inequality LP to standard form by adding slack variables."""
    if A_ineq is None:
        return c, np.zeros((0, len(c))), np.zeros(0)
    m, n = A_ineq.shape
    A = np.hstack([A_ineq, np.eye(m)])
    c_std = np.hstack([c, np.zeros(m)])
    return c_std, A, b_ineq.copy()

def initial_point(A, b, c):
    """Initialize using least-squares projection (Nocedal & Wright, 2006)."""
    n, m = len(c), A.shape[0]
    x = np.ones(n)
    s = np.ones(n)
    
    if m > 0:
        rhs = b - A.dot(x)
        try:
            AAT = A.dot(A.T)
            if np.linalg.cond(AAT) < 1e10:
                lam = np.linalg.solve(AAT, rhs)
                x += A.T.dot(lam)
        except:
            pass
    
    return np.maximum(x, 1e-4), np.zeros(m), np.maximum(s, 1e-4)

def compute_residuals(A, b, c, x, y, s):
    """Compute KKT residuals."""
    return A.T.dot(y) + s - c, A.dot(x) - b, x * s

def mu_val(x, s):
    """Duality measure (average complementarity)."""
    return np.dot(x, s) / len(x)

In [4]:
def solve_pd_system(A, x, s, y, rc, rb, sigma, mu):
    """Solve primal-dual system via normal equations with regularization."""
    n = len(x)
    s_safe = np.maximum(s, 1e-12)
    x_safe = np.maximum(x, 1e-12)
    
    Sinv = np.diag(1.0 / s_safe)
    D2 = np.diag(x_safe / s_safe)
    
    rhs_xs = -(x_safe * s_safe - sigma * mu)
    M = A.dot(D2).dot(A.T)
    rhs_y = -rb - A.dot(Sinv).dot(x_safe * rc + rhs_xs)
    
    reg = 1e-10 * np.eye(M.shape[0]) if M.shape[0] > 0 else np.zeros((0, 0))
    try:
        dy = np.linalg.solve(M + reg, rhs_y) if M.shape[0] > 0 else np.zeros(0)
    except np.linalg.LinAlgError:
        dy, *_ = np.linalg.lstsq(M + reg, rhs_y, rcond=None)
    
    ds = -rc - A.T.dot(dy)
    dx = -Sinv.dot(rhs_xs + x_safe * ds)
    return dx, dy, ds

def step_length(z, dz, tau=0.995):
    """Maximum step maintaining positivity."""
    alpha = 1.0
    neg = dz < 0
    if np.any(neg):
        alpha = min(1.0, tau * np.min(-z[neg] / dz[neg]))
    return max(alpha, 1e-12)

def central_path_fixed(A, b, c, x0, y0, s0, alpha=0.9, sigma=0.5, max_iters=100, tol=1e-8):
    """Algorithm 1: Fixed step-size central path method."""
    x, y, s = x0.copy(), y0.copy(), s0.copy()
    hist = {'obj': [], 'mu': [], 'gap': [], 'x': [], 's': []}
    
    for k in range(max_iters):
        rc, rb, _ = compute_residuals(A, b, c, x, y, s)
        mu = mu_val(x, s)
        gap = c.dot(x) - b.dot(y)
        
        hist['obj'].append(c.dot(x))
        hist['mu'].append(max(mu, 1e-16))  # Prevent log(0)
        hist['gap'].append(max(abs(gap), 1e-16))
        hist['x'].append(x.copy())
        hist['s'].append(s.copy())
        
        if abs(gap) < tol and mu < tol:
            break
        
        dx, dy, ds = solve_pd_system(A, x, s, y, rc, rb, sigma, mu)
        alpha_k = min(step_length(x, dx, alpha), step_length(s, ds, alpha))
        
        x += alpha_k * dx
        y += alpha_k * dy
        s += alpha_k * ds
    
    return hist

In [5]:
def central_path_adaptive(A, b, c, x0, y0, s0, max_iters=100, tol=1e-8):
    """Algorithm 2: Adaptive central path with predictor-based centering."""
    x, y, s = x0.copy(), y0.copy(), s0.copy()
    hist = {'obj': [], 'mu': [], 'gap': [], 'sigma': [], 'x': [], 's': []}
    
    for k in range(max_iters):
        rc, rb, _ = compute_residuals(A, b, c, x, y, s)
        mu = mu_val(x, s)
        gap = c.dot(x) - b.dot(y)
        
        hist['obj'].append(c.dot(x))
        hist['mu'].append(max(mu, 1e-16))
        hist['gap'].append(max(abs(gap), 1e-16))
        hist['x'].append(x.copy())
        hist['s'].append(s.copy())
        
        if abs(gap) < tol and mu < tol:
            break
        
        # Affine predictor
        dx_aff, dy_aff, ds_aff = solve_pd_system(A, x, s, y, rc, rb, 0.0, mu)
        alpha_aff = min(step_length(x, dx_aff, 0.99), step_length(s, ds_aff, 0.99))
        
        mu_aff = mu_val(x + alpha_aff * dx_aff, s + alpha_aff * ds_aff)
        sigma = (mu_aff / mu) ** 3 if mu > 1e-12 else 0.1
        sigma = np.clip(sigma, 1e-6, 1.0)
        hist['sigma'].append(sigma)
        
        # Corrector
        dx, dy, ds = solve_pd_system(A, x, s, y, rc, rb, sigma, mu)
        alpha_k = min(step_length(x, dx, 0.995), step_length(s, ds, 0.995))
        
        x += alpha_k * dx
        y += alpha_k * dy
        s += alpha_k * ds
    
    return hist

In [6]:
def mehrotra(A, b, c, x0, y0, s0, max_iters=50, tol=1e-8):
    """Algorithm 3: Mehrotra's predictor-corrector."""
    x, y, s = x0.copy(), y0.copy(), s0.copy()
    n = len(x)
    hist = {'obj': [], 'mu': [], 'gap': [], 'sigma': [], 'x': [], 's': []}
    
    for k in range(max_iters):
        rc, rb, _ = compute_residuals(A, b, c, x, y, s)
        mu = mu_val(x, s)
        gap = c.dot(x) - b.dot(y)
        
        hist['obj'].append(c.dot(x))
        hist['mu'].append(max(mu, 1e-16))
        hist['gap'].append(max(abs(gap), 1e-16))
        hist['x'].append(x.copy())
        hist['s'].append(s.copy())
        
        if abs(gap) < tol and mu < tol:
            break
        
        # PREDICTOR
        dx_aff, dy_aff, ds_aff = solve_pd_system(A, x, s, y, rc, rb, 0.0, mu)
        alpha_pri = step_length(x, dx_aff, 0.99)
        alpha_dual = step_length(s, ds_aff, 0.99)
        
        mu_aff = mu_val(x + alpha_pri * dx_aff, s + alpha_dual * ds_aff)
        sigma = (mu_aff / mu) ** 3 if mu > 1e-12 else 0.1
        sigma = np.clip(sigma, 1e-6, 1.0)
        hist['sigma'].append(sigma)
        
        # CORRECTOR
        dx, dy, ds = solve_pd_system(A, x, s, y, rc, rb, sigma, mu)
        
        # Second-order correction
        correction = dx_aff * ds_aff
        s_safe = np.maximum(s, 1e-12)
        dx -= correction / s_safe
        
        alpha_pri = step_length(x, dx, 0.995)
        alpha_dual = step_length(s, ds, 0.995)
        
        x += alpha_pri * dx
        y += alpha_dual * dy
        s += alpha_dual * ds
    
    return hist

In [7]:
problems = {}

# LP1: Simple 2D (visualization)
c1, A1, b1 = make_standard_form(
    np.array([-1.1, -1.0]),
    np.array([[1.0, 1.0]]),
    np.array([6.0])
)
problems['LP1_Simple2D'] = (c1, A1, b1, "Simple 2D Problem")

# LP2: Resource allocation
c2, A2, b2 = make_standard_form(
    np.array([-30.0, -20.0]),
    np.array([[2.0, 1.0], [1.0, 3.0]]),
    np.array([8.0, 8.0])
)
problems['LP2_Resource'] = (c2, A2, b2, "Resource Allocation")

# LP3: Diet problem
c3, A3, b3 = make_standard_form(
    np.array([2.0, 3.0]),
    np.array([[-1.0, -2.0], [-3.0, -1.0]]),
    np.array([-4.0, -3.0])
)
problems['LP3_Diet'] = (c3, A3, b3, "Diet Problem")

print(f"Loaded {len(problems)} test problems\n")

Loaded 3 test problems



In [8]:
def create_main_figure(results_dict):
    """
    Create THE main figure for the paper: 2x3 grid showing:
    - Row 1: Duality gap convergence for all 3 problems
    - Row 2: Duality measure μ convergence for all 3 problems
    """
    fig = plt.figure(figsize=(11, 6))
    gs = fig.add_gridspec(2, 3, hspace=0.35, wspace=0.30)
    
    problem_names = ['LP1_Simple2D', 'LP2_Resource', 'LP3_Diet']
    titles = ['2D Problem', 'Resource Allocation', 'Diet Problem']
    
    for col, (name, title) in enumerate(zip(problem_names, titles)):
        if name not in results_dict:
            continue
        
        res = results_dict[name]
        
        # Top row: Duality gap
        ax1 = fig.add_subplot(gs[0, col])
        for method in ['fixed', 'adaptive', 'mehrotra']:
            h = res[method]
            if len(h['gap']) > 0:
                gaps = np.array(h['gap'])
                ax1.semilogy(gaps, 
                           color=COLORS[method],
                           marker=MARKERS[method],
                           markersize=3,
                           markevery=max(1, len(gaps)//8),
                           label=method.capitalize(),
                           linewidth=1.5)
        
        ax1.axhline(y=1e-8, color=COLORS['tolerance'], 
                   linestyle=':', linewidth=1.2, label='Tolerance', alpha=0.7)
        ax1.set_xlabel('Iteration')
        ax1.set_ylabel('Duality Gap' if col == 0 else '')
        ax1.set_title(f'({chr(97+col)}) {title}')
        ax1.grid(True, which='both', alpha=0.3)
        if col == 2:
            ax1.legend(loc='best', fontsize=7, frameon=True)
        ax1.set_ylim(bottom=1e-12)
        
        # Bottom row: μ convergence
        ax2 = fig.add_subplot(gs[1, col])
        for method in ['fixed', 'adaptive', 'mehrotra']:
            h = res[method]
            if len(h['mu']) > 0:
                mu_vals = np.array(h['mu'])
                ax2.semilogy(mu_vals,
                           color=COLORS[method],
                           marker=MARKERS[method],
                           markersize=3,
                           markevery=max(1, len(mu_vals)//8),
                           label=method.capitalize(),
                           linewidth=1.5)
        
        ax2.axhline(y=1e-8, color=COLORS['tolerance'],
                   linestyle=':', linewidth=1.2, alpha=0.7)
        ax2.set_xlabel('Iteration')
        ax2.set_ylabel('Duality Measure μ' if col == 0 else '')
        ax2.grid(True, which='both', alpha=0.3)
        ax2.set_ylim(bottom=1e-12)
    
    plt.savefig('../figures/Fig1_main_convergence.pdf')
    plt.savefig('../figures/Fig1_main_convergence.png', dpi=300)
    plt.close()
    print("✓ Created Fig 1: Main convergence comparison")

In [9]:
def create_trajectory_figure(results_dict):
    """Central path visualization for 2D problems."""
    fig, axes = plt.subplots(1, 3, figsize=(11, 3.5))
    
    problem_names = ['LP1_Simple2D', 'LP2_Resource', 'LP3_Diet']
    titles = ['2D Problem', 'Resource Allocation', 'Diet Problem']
    
    for idx, (name, title, ax) in enumerate(zip(problem_names, titles, axes)):
        if name not in results_dict:
            continue
        
        res = results_dict[name]
        
        for method in ['fixed', 'adaptive', 'mehrotra']:
            h = res[method]
            if len(h['x']) > 0 and len(h['x'][0]) >= 2:
                x1 = np.array([x[0] for x in h['x']])
                x2 = np.array([x[1] for x in h['x']])
                
                ax.plot(x1, x2,
                       color=COLORS[method],
                       marker=MARKERS[method],
                       markersize=4,
                       markevery=max(1, len(x1)//6),
                       label=method.capitalize(),
                       linewidth=1.5,
                       alpha=0.8)
                
                # Mark start (only once)
                if method == 'fixed':
                    ax.scatter([x1[0]], [x2[0]], 
                             c='black', s=120, marker='*',
                             label='Initial', zorder=5, 
                             edgecolors='white', linewidths=1)
                
                # Mark end
                ax.scatter([x1[-1]], [x2[-1]],
                         c=COLORS[method], s=80, marker='X',
                         zorder=5, edgecolors='white', linewidths=1)
        
        ax.set_xlabel('$x_1$')
        ax.set_ylabel('$x_2$' if idx == 0 else '')
        ax.set_title(f'({chr(97+idx)}) {title}')
        ax.grid(True, alpha=0.3)
        if idx == 2:
            ax.legend(loc='best', fontsize=7)
        ax.set_aspect('equal', adjustable='datalim')
    
    plt.tight_layout()
    plt.savefig('../figures/Fig2_trajectories.pdf')
    plt.savefig('../figures/Fig2_trajectories.png', dpi=300)
    plt.close()
    print("✓ Created Fig 2: Central path trajectories")

In [10]:
def create_results_table(results_dict):
    """Generate LaTeX table of convergence results."""
    with open('../figures/Table1_results.tex', 'w') as f:
        f.write('\\begin{table}[t]\n')
        f.write('\\centering\n')
        f.write('\\caption{Convergence comparison of interior point methods}\n')
        f.write('\\label{tab:results}\n')
        f.write('\\begin{tabular}{llccc}\n')
        f.write('\\hline\n')
        f.write('Problem & Method & Iterations & Final Obj. & Final $\\mu$ \\\\\n')
        f.write('\\hline\n')
        
        for name, res in results_dict.items():
            title = res['title']
            f.write(f'\\multirow{{4}}{{*}}{{{title}}} \n')
            
            for method in ['fixed', 'adaptive', 'mehrotra']:
                h = res[method]
                iters = len(h['obj'])
                obj = h['obj'][-1] if h['obj'] else 0
                mu = h['mu'][-1] if h['mu'] else 0
                f.write(f'  & {method.capitalize():12s} & {iters:3d} & '
                       f'{obj:10.4f} & {mu:.2e} \\\\\n')
            
            if res.get('scipy_obj'):
                f.write(f'  & {"SciPy":12s} & {"--":>3s} & '
                       f'{res["scipy_obj"]:10.4f} & {"--":>8s} \\\\\n')
            
            f.write('\\hline\n')
        
        f.write('\\end{tabular}\n')
        f.write('\\end{table}\n')
    
    print("✓ Created Table 1: Numerical results (LaTeX)")

In [11]:
def create_appendix_figures(results_dict):
    """Generate supplementary figures for appendix - showing distance to optimum."""
    
    for name, res in results_dict.items():
        # Use scipy as reference optimal value
        opt_val = res.get('scipy_obj')
        if opt_val is None:
            # Use best achieved value as reference
            opt_val = min(
                res['mehrotra']['obj'][-1],
                res['adaptive']['obj'][-1],
                res['fixed']['obj'][-1]
            )
        
        fig, ax = plt.subplots(figsize=(6, 4))
        
        # Find maximum iteration where any method is still converging
        max_interesting_iter = 0
        
        for method in ['fixed', 'adaptive', 'mehrotra']:
            h = res[method]
            if len(h['obj']) > 0:
                # Compute distance to optimum
                obj_vals = np.array(h['obj'])
                distance = np.abs(obj_vals - opt_val)
                distance = np.maximum(distance, 1e-16)
                
                # Find last iteration where distance > 10 * tolerance
                converged = np.where(distance < 1e-7)[0]
                if len(converged) > 0:
                    last_iter = converged[0] + 5  # Add 5 iterations buffer
                else:
                    last_iter = len(distance)
                max_interesting_iter = max(max_interesting_iter, last_iter)
        
        # Cap at reasonable maximum
        max_interesting_iter = min(max_interesting_iter, 50)
        
        for method in ['fixed', 'adaptive', 'mehrotra']:
            h = res[method]
            if len(h['obj']) > 0:
                # Compute distance to optimum
                obj_vals = np.array(h['obj'])
                distance = np.abs(obj_vals - opt_val)
                distance = np.maximum(distance, 1e-16)
                
                # Plot only interesting iterations
                plot_iters = min(len(distance), max_interesting_iter)
                iters = np.arange(plot_iters)
                
                ax.semilogy(iters, distance[:plot_iters],
                           color=COLORS[method],
                           marker=MARKERS[method],
                           markersize=5,
                           markevery=max(1, plot_iters//10),
                           label=method.capitalize(),
                           linewidth=2.0,
                           alpha=0.9)
        
        ax.axhline(y=1e-8, color=COLORS['tolerance'],
                  linestyle=':', linewidth=1.5, label='Tolerance (10⁻⁸)', alpha=0.7)
        
        ax.set_xlabel('Iteration', fontweight='bold')
        ax.set_ylabel('|f(xₖ) - f(x*)|', fontweight='bold', fontsize=11)
        ax.set_title(f'{res["title"]} - Distance to Optimum', 
                    fontweight='bold', fontsize=11)
        ax.legend(loc='best', fontsize=9, frameon=True)
        ax.grid(True, which='both', alpha=0.3)
        ax.set_ylim(bottom=1e-14, top=1e2)
        
        # Set x-axis limit to show only interesting region
        ax.set_xlim(left=-0.5, right=max_interesting_iter)
        
        plt.tight_layout()
        plt.savefig(f'../figures/AppendixA_{name}_distance.pdf')
        plt.savefig(f'../figures/AppendixA_{name}_distance.png', dpi=300)
        plt.close()
    
    print("✓ Created Appendix A: Distance-to-optimum plots")

In [12]:
def run_experiments():
    """Execute all algorithms and generate publication figures."""
    results = {}
    
    print("=" * 70)
    print("RUNNING INTERIOR POINT METHOD EXPERIMENTS")
    print("=" * 70 + "\n")
    
    for name, (c, A, b, title) in problems.items():
        print(f"Solving: {title}")
        
        x0, y0, s0 = initial_point(A, b, c)
        
        # Run algorithms
        h_fix = central_path_fixed(A, b, c, x0, y0, s0, max_iters=200)
        h_ad = central_path_adaptive(A, b, c, x0, y0, s0, max_iters=200)
        h_meh = mehrotra(A, b, c, x0, y0, s0, max_iters=100)
        
        # SciPy reference
        try:
            res = linprog(c, A_eq=A, b_eq=b, bounds=[(0, None)] * len(c),
                         method='highs-ipm', options={'disp': False})
            scipy_obj = float(res.fun) if res.success else None
        except:
            scipy_obj = None
        
        results[name] = {
            'fixed': h_fix,
            'adaptive': h_ad,
            'mehrotra': h_meh,
            'scipy_obj': scipy_obj,
            'title': title
        }
        
        print(f"  Fixed:    {len(h_fix['obj']):3d} iters, "
              f"obj={h_fix['obj'][-1]:9.4f}, μ={h_fix['mu'][-1]:.2e}")
        print(f"  Adaptive: {len(h_ad['obj']):3d} iters, "
              f"obj={h_ad['obj'][-1]:9.4f}, μ={h_ad['mu'][-1]:.2e}")
        print(f"  Mehrotra: {len(h_meh['obj']):3d} iters, "
              f"obj={h_meh['obj'][-1]:9.4f}, μ={h_meh['mu'][-1]:.2e}")
        if scipy_obj:
            print(f"  SciPy:          obj={scipy_obj:9.4f}")
        print()
    
    # Generate publication figures
    print("\nGenerating publication figures...")
    create_main_figure(results)
    create_trajectory_figure(results)
    create_results_table(results)
    create_appendix_figures(results)
    
    # Save summary JSON
    summary = {}
    for name, res in results.items():
        summary[name] = {
            'title': res['title'],
            'iterations': {m: len(res[m]['obj']) for m in ['fixed', 'adaptive', 'mehrotra']},
            'final_objective': {m: float(res[m]['obj'][-1]) for m in ['fixed', 'adaptive', 'mehrotra']},
            'scipy': res['scipy_obj']
        }
    
    with open('../figures/summary.json', 'w') as f:
        json.dump(summary, f, indent=2)
    
    print("\n" + "=" * 70)
    print("EXPERIMENTS COMPLETE")
    print("=" * 70)
    print("\nGenerated files:")
    print("  Main paper:")
    print("    - Fig1_main_convergence.pdf (use this!)")
    print("    - Fig2_trajectories.pdf")
    print("    - Table1_results.tex")
    print("  Appendix:")
    print("    - AppendixA_*_objective.png (3 files)")
    print("  Summary:")
    print("    - summary.json")
    print("=" * 70 + "\n")
    
    return results

In [13]:
results = run_experiments()
print('Done. Figures saved to figures/ directory.')

RUNNING INTERIOR POINT METHOD EXPERIMENTS

Solving: Simple 2D Problem
  Fixed:    200 iters, obj=  -3.6745, μ=2.00e+00
  Adaptive: 200 iters, obj=  -4.8730, μ=1.22e+00
  Mehrotra: 100 iters, obj= -10.7193, μ=1.53e+13
  SciPy:          obj=  -6.6000

Solving: Resource Allocation
  Fixed:    200 iters, obj=-178.6101, μ=1.24e+00
  Adaptive: 200 iters, obj=-147.9539, μ=1.09e+00
  Mehrotra: 100 iters, obj= -69.2311, μ=8.74e+14
  SciPy:          obj=-128.0000

Solving: Diet Problem
  Fixed:    200 iters, obj=  41.4757, μ=1.24e+00
  Adaptive: 200 iters, obj=  15.6714, μ=6.92e-01
  Mehrotra: 100 iters, obj=  35.0447, μ=4.31e+00
  SciPy:          obj=   6.2000


Generating publication figures...
✓ Created Fig 1: Main convergence comparison
✓ Created Fig 2: Central path trajectories
✓ Created Table 1: Numerical results (LaTeX)
✓ Created Appendix A: Distance-to-optimum plots

EXPERIMENTS COMPLETE

Generated files:
  Main paper:
    - Fig1_main_convergence.pdf (use this!)
    - Fig2_trajectories.p